## TOC:
* [Artists](#Artists)
* [Generator](#Generator)
* [Rhymer](#Rhymer)

# All Imports

In [1]:
# package imports
# !pip install -q bs4 # uncomment this line to install
from bs4 import BeautifulSoup # easy webscraping
import requests # for sending http requests
import re # regular expressions, excellent for string searching/replacing, 
# extremely powerful for manual NLP, check out the documentation at https://docs.python.org/3/library/re.html
import os  # accessing file paths

If you have ongoing issues with the transformers library, Also consider installing from source: 

git clone https://github.com/huggingface/transformers.git 

cd transformers (in Dropbox/C/25_Studium_Master/TUM_RCI/2021_WS/NLP/3-Exercises-Workspace/transformers)

pip install -e

Also update git


In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1" to deactive 

import tensorflow as tf

# Update Transformers from the source if you get errors:
#!pip install git+https://github.com/huggingface/transformers

from transformers import AutoConfig, AutoTokenizer, TFAutoModelForCausalLM, default_data_collator, pipeline 
from transformers import GPT2Model, GPT2Config
# DefaultDataCollator: for the error https://github.com/huggingface/transformers/pull/5015
from official.nlp import optimization
!pip install -q datasets
from datasets import Dataset  # https://huggingface.co/docs/datasets/

from bs4 import BeautifulSoup 
import requests 
import re 
import json
import random
from collections import Counter
import math

model_name = "distilgpt2" # select a pretrained language model from https://huggingface.co/models

# gtp3 is huge, needed several GPus for traning, was in the news, soendt fit in to small gpus so we use gtp2 

In [3]:
!pip install GPUtil
import GPUtil
from numba import cuda
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 | 54% | 19% |


# Artists <a class="anchor" id="Artists"></a>



## Blues

In [21]:
# insert a link to the lyricsfreak artist page of your choice
blues_artist_links = [
    "https://www.lyricsfreak.com/b/bb+king/",
    "https://www.lyricsfreak.com/r/robert+johnson/",
    "https://www.lyricsfreak.com/m/muddy+waters/",
    "https://www.lyricsfreak.com/b/buddy+guy/",  
    "https://www.lyricsfreak.com/e/eric+clapton/"
]


## Country

In [22]:
# insert a link to the lyricsfreak artist page of your choice
country_artist_links = [
    "https://www.lyricsfreak.com/j/johnny+cash/",
    "https://www.lyricsfreak.com/t/tim+mcgraw/",
    "https://www.lyricsfreak.com/g/george+strait/",
    "https://www.lyricsfreak.com/c/carter+family/",
    "https://www.lyricsfreak.com/d/dolly+parton/"
]


# Jazz

In [23]:
# insert a link to the lyricsfreak artist page of your choice
jazz_artist_links = [
    "https://www.lyricsfreak.com/l/louis+armstrong/",
    "https://www.lyricsfreak.com/e/ella+fitzgerald/",
    "https://www.lyricsfreak.com/m/cole+porter/",
    "https://www.lyricsfreak.com/r/ray+charles/",
    "https://www.lyricsfreak.com/n/nina+simone/"
]


# Metal

In [24]:
# insert a link to the lyricsfreak artist page of your choice
metal_artist_links = [
    "https://www.lyricsfreak.com/b/black+sabbath/",
    "https://www.lyricsfreak.com/m/metallica/",
    "https://www.lyricsfreak.com/m/megadeth/",
    "https://www.lyricsfreak.com/d/dio/",
    "https://www.lyricsfreak.com/t/tool/"
]


# Pop

In [25]:
# insert a link to the lyricsfreak artist page of your choice
pop_artist_links = [
    "https://www.lyricsfreak.com/b/beatles/",
    "https://www.lyricsfreak.com/m/madonna/",
    "https://www.lyricsfreak.com/e/elton+john/",
    "https://www.lyricsfreak.com/m/michael+jackson/",
    "https://www.lyricsfreak.com/s/rihanna/"
]


# Rock

In [26]:
# insert a link to the lyricsfreak artist page of your choice
rock_artist_links = [
    "https://www.lyricsfreak.com/b/rolling+stones/",
    "https://www.lyricsfreak.com/n/nirvana/",
    "https://www.lyricsfreak.com/l/led+zeppelin/", # find link!
    "https://www.lyricsfreak.com/a/ac+dc/",
    "https://www.lyricsfreak.com/q/queen/"
]


# Download all

In [10]:
# helper functions
def get_links(artist_link):
    page = requests.get(artist_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    a = soup.findAll("a", {"class": "lf-link lf-link--primary"})
    links = [i["href"] for i in a]
    print(str(len(links)) + " lyrics found at " + artist_link)
    return(links)

def get_lyrics(links):
    lyrics = {}
    for l in links:
        page = requests.get("https://www.lyricsfreak.com" + str(l))
        soup = BeautifulSoup(page.content, 'html.parser')
        title = re.sub("\+", "_", l.split("/")[-1].split("_")[0])
        lyrics[title] = soup.find(id = "content")
    return(lyrics)

# some strings in the lyrics that will be removed
beg = re.compile('<div class.*')
end = re.compile('</div>')
lin = re.compile('<br/>')
cho = re.compile('chorus|verse|intro', flags = re.IGNORECASE)
cr = re.compile('\r')
patterns = [beg, end, lin, cho, cr]

def clean_text(patterns, text):
    text = str(text)
    for pattern in patterns:
        text = re.sub(pattern, "", text)
    return(text.strip())
        
lyrics_dict = {}

output_path = "/home/volkan/Dropbox/C/25_Studium_Master/TUM_RCI/2021_WS/NLP/3-Exercises-Workspace/EchoCanyon/lyrics/"

genre_links = [blues_artist_links, country_artist_links, jazz_artist_links, metal_artist_links, rock_artist_links, pop_artist_links]
genre_names = ["blues", "country", "jazz", "metal", "rock", "pop"]
for count, genres in enumerate(genre_links):
    
    lyrics_links = None
    lyrics = None
    for artist_link in genres:

        lyrics_links = [link for link in (get_links(artist_link))]
        lyrics = get_lyrics(lyrics_links)
        lyrics = {title: clean_text(patterns,lyric) for title, lyric in lyrics.items()}

        lyrics_dict[re.sub("\+", "_", artist_link.split("/")[-2])] = lyrics

    with open(output_path + genre_names[count] +"_lyrics.json", "w", encoding = "utf8") as f:

        json.dump(lyrics_dict, f)
    
    lyrics_dict = {}
    


200 lyrics found at https://www.lyricsfreak.com/b/bb+king/
61 lyrics found at https://www.lyricsfreak.com/r/robert+johnson/
112 lyrics found at https://www.lyricsfreak.com/m/muddy+waters/
111 lyrics found at https://www.lyricsfreak.com/b/buddy+guy/
200 lyrics found at https://www.lyricsfreak.com/e/eric+clapton/
235 lyrics found at https://www.lyricsfreak.com/j/johnny+cash/
209 lyrics found at https://www.lyricsfreak.com/t/tim+mcgraw/
200 lyrics found at https://www.lyricsfreak.com/g/george+strait/
40 lyrics found at https://www.lyricsfreak.com/c/carter+family/
200 lyrics found at https://www.lyricsfreak.com/d/dolly+parton/
205 lyrics found at https://www.lyricsfreak.com/l/louis+armstrong/
200 lyrics found at https://www.lyricsfreak.com/e/ella+fitzgerald/
176 lyrics found at https://www.lyricsfreak.com/m/cole+porter/
200 lyrics found at https://www.lyricsfreak.com/r/ray+charles/
206 lyrics found at https://www.lyricsfreak.com/n/nina+simone/
164 lyrics found at https://www.lyricsfreak.co

# Download my Artists

In [4]:
# FROM: 10 Finetuning a Language Model with Huggingface - Lyrics Mashup
# CHANGES: Changed the directories to save genres separately 

# I used the my own machine here to save time during the downloading of lyrics, 
# That is why the above download paths are for my own PC instead of drive. 
# This cell and the above 6 needs to be ran own your own machine as your CPU is 
# Faster then Google Colabs GPU for downloads. Then the files need to be copied
# to drive afterwards and the drive has to be mounted as it's done at the start
# of the notebook 

# Commented out to make the restarting and runnning easier as this is needed once 



# helper functions
# insert links to lyricsfreak artist pages
artist_links = [
    "https://www.lyricsfreak.com/b/blues+brothers/"
]

# helper functions
def get_links(artist_link):
    page = requests.get(artist_link)
    soup = BeautifulSoup(page.content, 'html.parser')
    a = soup.findAll("a", {"class": "lf-link lf-link--primary"})
    links = [i["href"] for i in a]
    print(str(len(links)) + " lyrics found at " + artist_link)
    return(links)

def get_lyrics(links):
    lyrics = {}
    for l in links:
        page = requests.get("https://www.lyricsfreak.com" + str(l))
        soup = BeautifulSoup(page.content, 'html.parser')
        title = re.sub("\+", "_", l.split("/")[-1].split("_")[0])
        lyrics[title] = soup.find(id = "content")
    return(lyrics)

# some strings in the lyrics that will be removed
beg = re.compile('<div class.*')
end = re.compile('</div>')
lin = re.compile('<br/>')
cho = re.compile('chorus|verse|intro', flags = re.IGNORECASE)
cr = re.compile('\r')
patterns = [beg, end, lin, cho, cr]

def clean_text(patterns, text):
    text = str(text)
    for pattern in patterns:
        text = re.sub(pattern, "", text)
    return(text.strip())
        
lyrics_dict = {}

output_path = "/home/volkan/Dropbox/C/25_Studium_Master/TUM_RCI/2021_WS/NLP/3-Exercises-Workspace/EchoCanyon/lyrics/"


for artist_link in artist_links:
    
    lyrics_links = [link for link in (get_links(artist_link))]
    lyrics = get_lyrics(lyrics_links)
    lyrics = {title: clean_text(patterns,lyric) for title, lyric in lyrics.items()}
    
    lyrics_dict[re.sub("\+", "_", artist_link.split("/")[-2])] = lyrics

with open(output_path + "my_artists.json", "w", encoding = "utf8") as f: ##### CHANGE HERE
    
    json.dump(lyrics_dict, f)

66 lyrics found at https://www.lyricsfreak.com/b/blues+brothers/
